Deep Learning
=============

Loading and Transforming Data
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [ ]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [ ]:
num_classes = 10

def extract(filename):
  tar = tarfile.open(filename)
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  print('Extracting data for %s. This may take a while. Please wait.' % root)
  sys.stdout.flush()
  tar.extractall()
  tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root)) if d != '.DS_Store']
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = extract(train_filename)
test_folders = extract(test_filename)

---
Random Display
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [ ]:
import os, random
Large = ['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
Small = ['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']
for L,S in zip(Large,Small):
    display(Image(filename = L+'/'+random.choice(os.listdir(L))))
    display(Image(filename = S+'/'+random.choice(os.listdir(S))))
    # Both above and below commands produce same result. Use bottom though.
    display(Image(filename =  os.path.join(L, random.choice(os.listdir(L)))))
    display(Image(filename =  os.path.join(S, random.choice(os.listdir(S)))))
    #print L, S

Now let's load the data in a more manageable format.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. The labels will be stored into a separate array of integers 0 through 9.

A few images might not be readable, we'll just skip them.

In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load(data_folders, min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size), dtype=np.float32)
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print(folder)
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          image_index, max_num_images))
      image_file = os.path.join(folder, image)
      try:
        # Normalize    
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth 
        #Shape Check
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :] = image_data
        labels[image_index] = label_index
        image_index += 1
      except IOError as e:
        print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  print('Labels:', labels.shape)
  return dataset, labels
train_dataset, train_labels = load(train_folders, 450000, 550000)
test_dataset, test_labels = load(test_folders, 18000, 20000)

---
Post Normalization Data
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [ ]:
# Run this cell multiple times.
%matplotlib inline  
i=0;
for i in range(0,10):
    index = random.randrange(0,train_dataset.shape[0])
    plt.matshow(train_dataset[index,:,:])
    plt.show()
    print i
### For some reason the above code works well in qtconsole launched eaerlier in this notebook.     

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
np.random.seed(133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

---
Balance across classes
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

Prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
train_size = 200000
valid_size = 10000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
train_labels = train_labels[valid_size:valid_size+train_size]
print('Training', train_dataset.shape, train_labels.shape)
print('Validation', valid_dataset.shape, valid_labels.shape)

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
import os, random
import pickle
pickle_file = 'notMNIST.pickle'
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

---
Remove Duplicates
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [2]:
import pickle
data = pickle.load(open("notMNIST.pickle","rb"))
test_dataset = data['test_dataset']
train_dataset = data['train_dataset']
valid_dataset = data['valid_dataset']
train_labels = data['train_labels']
test_labels = data['test_labels']
valid_labels = data['valid_labels']

In [3]:
import numpy as np
import hashlib
import time
from itertools import combinations

start = time.time()

same_set = []
d_sets = [train_dataset, test_dataset, valid_dataset]
d_sets_names = ["train_set", "test_set", "valid_set"]
rep_counts = {}

for i,x in enumerate(d_sets):
    same_set.append(set([hashlib.sha1(image_array).hexdigest() for image_array in x]))
    # above counts unique elements by assigning a hash. 
    rep_counts[d_sets_names[i]]= {"orig_len": len(x),
                                  "uniq_len": len(same_set[i]),
                                  "repeat_count" : len(x)-len(same_set[i])}
    
set_pairs = combinations(d_sets,2)
set_pair_names = combinations(d_sets_names,2)

for i, ((first_set_name, second_set_name),(first_set, second_set)) in enumerate(zip(set_pair_names,set_pairs)):
    print first_set_name, second_set_name
    total_set_lengths = rep_counts[first_set_name]["orig_len"] + rep_counts[second_set_name]["orig_len"]
    total_unique_lens = rep_counts[first_set_name]["uniq_len"] + rep_counts[second_set_name]["uniq_len"]
    final_unique_lens = len(set([hashlib.sha1(image_array).hexdigest() 
                                 for image_array in np.vstack((first_set,second_set))]))

    rep_counts[first_set_name + " " + second_set_name] = {"total_set_lengths": total_set_lengths,
                                                         "total_unique_lens": total_unique_lens,
                                                          "repeat_count": total_unique_lens - final_unique_lens}
for x in [[key,val]for key,val in rep_counts.items()]:
    print(x)
print(time.time()-start,"s")


train_set test_set
train_set valid_set
test_set valid_set
['test_set', {'repeat_count': 492, 'orig_len': 18724, 'uniq_len': 18232}]
['valid_set', {'repeat_count': 138, 'orig_len': 10000, 'uniq_len': 9862}]
['train_set valid_set', {'total_set_lengths': 210000, 'repeat_count': 972, 'total_unique_lens': 197414}]
['test_set valid_set', {'total_set_lengths': 28724, 'repeat_count': 130, 'total_unique_lens': 28094}]
['train_set', {'repeat_count': 12448, 'orig_len': 200000, 'uniq_len': 187552}]
['train_set test_set', {'total_set_lengths': 218724, 'repeat_count': 2094, 'total_unique_lens': 205784}]
(6.541959047317505, 's')


---
Off-the-Shlef (Benchmark)
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, zero_one_loss

train_sizes = [50, 5000, 10000,200000]
#train_sizes = [200000]

for N in train_sizes:
    print '\nTraining on %d examples' % N
    # For all the data (N=200000) use the SGD classifier
    model = SGDClassifier(random_state=111)#, multi_class='multinomial', solver='newton-cg')
    #model = LogisticRegression(random_state=111)#, solver='newton-cg')
    
    model.fit(train_dataset[:N].reshape((N, -1)), train_labels[:N])
    
    pred = model.predict(test_dataset.reshape((test_dataset.shape[0], -1)))
    
    error_rate = zero_one_loss(test_labels, pred)
    print 'Error rate: %.2f%%' % (error_rate*100.0)
    print classification_report(test_labels, pred, target_names=list('ABCDEFGHIJ'))


Training on 50 examples
Error rate: 48.67%
             precision    recall  f1-score   support

          A       0.49      0.75      0.59      1872
          B       0.50      0.73      0.59      1873
          C       0.60      0.27      0.37      1873
          D       0.88      0.40      0.56      1873
          E       0.45      0.75      0.56      1873
          F       0.72      0.49      0.58      1872
          G       0.50      0.76      0.60      1872
          H       0.61      0.20      0.30      1872
          I       0.08      0.05      0.06      1872
          J       0.59      0.72      0.65      1872

avg / total       0.54      0.51      0.49     18724


Training on 5000 examples
Error rate: 15.23%
             precision    recall  f1-score   support

          A       0.81      0.87      0.84      1872
          B       0.88      0.84      0.86      1873
          C       0.92      0.88      0.90      1873
          D       0.92      0.88      0.90      1873
     